In [1]:
!pip install html5lib
!pip install tqdm
!pip install pandas
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import html5lib


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Rismosch\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Rismosch\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Rismosch\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
url = "https://us.soccerway.com/international/world/world-cup/2022-qatar/group-stage/r49519/"
group_div = "block_competition_tables_group-wrapper"

my_agent = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15"}
response = requests.get(url, headers = my_agent)
soup = BeautifulSoup(response.text)

all_current = soup.find_all("div", {"class": group_div})

team_list = []

for div in all_current:
    
    tables = div.findChildren("table", recursive=True)
    table = tables[0]
    
    trs = table.findChildren("tr", recursive=True)

    for tr in trs:
        tds = tr.findChildren("td", recursive=True)
        for td in tds:
            classes = td.get("class")
            if 'large-link' in classes:
                team_name = td.findChildren("a")[0].contents[0]
                href = td.findChildren("a")[0]["href"]
                
                base_url = "https://us.soccerway.com/"
                link = f'{base_url}/{href}'
                
                team = {
                    "name": team_name,
                    "link": link,
                }
                
                team_list.append(team)
            
data_frame = pd.DataFrame(data=team_list)
print(data_frame)

              name                                               link
0      Netherlands  https://us.soccerway.com///teams/netherlands/n...
1          Ecuador  https://us.soccerway.com///teams/ecuador/ecuad...
2          Senegal  https://us.soccerway.com///teams/senegal/seneg...
3            Qatar  https://us.soccerway.com///teams/qatar/qatar/1...
4          England  https://us.soccerway.com///teams/england/engla...
5             Iran   https://us.soccerway.com///teams/iran/iran/1178/
6    United States  https://us.soccerway.com///teams/united-states...
7            Wales  https://us.soccerway.com///teams/wales/wales/2...
8     Saudi Arabia  https://us.soccerway.com///teams/saudi-arabia/...
9           Poland  https://us.soccerway.com///teams/poland/poland...
10          Mexico  https://us.soccerway.com///teams/mexico/mexico...
11       Argentina  https://us.soccerway.com///teams/argentina/arg...
12          France  https://us.soccerway.com///teams/france/france...
13         Tunisia  

In [4]:
first_team = team_list[0]

match = f'{first_team["link"]}matches/'

print(match)

https://us.soccerway.com///teams/netherlands/netherlands/1552/matches/
